In [9]:
import matplotlib.pyplot as plt
from dataload import *
from gan_architecture import *
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataload import *
from gan_architecture import *
import pandas as pd
import numpy as np
import copy

In [10]:
class SuperNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 4)
        self.bn = nn.BatchNorm1d(4)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.lin(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [11]:
bs = 16
input = torch.full((bs, 4), 1.5, dtype=torch.float, device='cpu')

In [12]:
s = SuperNetwork(4)
s2 = copy.deepcopy(s)
criterion = nn.BCELoss() 
optimizer = optim.Adam(s.parameters(), lr = 0.002)

In [13]:
# print(input)

In [14]:
loss = criterion(s(input), input)


In [15]:
for param in s.parameters():
  print(param.data)

for param in s2.parameters():
  print(param.data)

tensor([[ 0.1225, -0.3776, -0.4937, -0.3117],
        [ 0.3008,  0.2439, -0.3037,  0.0155],
        [ 0.2725, -0.0426,  0.0411,  0.0291],
        [-0.0056,  0.3533, -0.1759, -0.4685]])
tensor([-0.4748,  0.1979,  0.3418, -0.4259])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])
tensor([[ 0.1225, -0.3776, -0.4937, -0.3117],
        [ 0.3008,  0.2439, -0.3037,  0.0155],
        [ 0.2725, -0.0426,  0.0411,  0.0291],
        [-0.0056,  0.3533, -0.1759, -0.4685]])
tensor([-0.4748,  0.1979,  0.3418, -0.4259])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])


In [28]:
s.modules()

<generator object Module.modules at 0x7f358004dc80>